In [4]:
import numpy as np
import pandas as pd

# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)\n",

from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [5]:
#creating geolocations for each hotel using Geopy
# source https://towardsdatascience.com/geocode-with-python-161ec1e62b89

# instatiate the Nominatim from Geopy
locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


###Ultimately not used below, once it was determine the error was unresolvable

# # 2- - create location column
# data['location'] = data['address'].apply(geocode)
# # 3 - create longitude, laatitude and altitude from location column (returns tuple)
# data['point'] = data['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# # 4 - split point column into latitude, longitude and altitude columns
# data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [6]:
# applying the geocode function above ran into errors that it could not resolve. 
# created the  function below to address this issue (no pun intended)

def get_geocodes(data):
    locationlist = []
    counter = 0
    for index, row in data.iterrows():
        #empty dictionary to hold the values of the lat and long
        geocodeadd = {}
        # this will gather geocode for specific hotel street address
        try:
            geocode = RateLimiter(locator.geocode, min_delay_seconds=5)
            location = locator.geocode(row['address'])  
#             locationlist.append(location)
            geocodeadd['name'] = row['name']
            geocodeadd['address'] = row['address']
            geocodeadd['latitude'] = location.latitude
            geocodeadd['longitude'] = location.longitude
            print(f"{counter}: Latitude = {location.latitude}, Longitude = {location.longitude}")
        # if the address of the hotel does not produce geocode the below will provide geocode for the zipcode
        except: 
            try:
                geocode = RateLimiter(locator.geocode, min_delay_seconds=5)
                location = locator.geocode(row['address2'])                 
            
                geocodeadd['name'] = row['name']
                geocodeadd['address'] = row['address']
                geocodeadd['latitude'] = location.latitude
                geocodeadd['longitude'] = location.longitude
                print(f"{counter}: Latitude = {location.latitude}, Longitude = {location.longitude}")
            # catch all for non producable geocodes
            except: 
                geocodeadd['name'] = row['name']
                geocodeadd['address'] = row['address']
                geocodeadd['latitude'] = 'N/A'
                geocodeadd['longitude'] = 'N/A'
                print(f"{counter}: Latitude = N/A, Longitude = N/A")
        locationlist.append(geocodeadd)
#         time.sleep(1)
        counter += 1
    df = pd.DataFrame(locationlist)
    final_df = pd.merge(data, df, on='name')
    return final_df

In [7]:
data = pd.read_csv('data_for_app6.csv')
testdata = data[:100]

In [8]:
testdata.head()

,Unnamed: 0,Unnamed: 0.1,rewards,brand,name,rating,description,street,city,state,postal,url,class_id,category,Bag_of_words,spacydocs,vectors,address,address2,vectormean
0,0,0,Hilton,Hampton Inn,Hampton Inn Alexander City,4.0,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,Hilton Hampton Inn We’re right off Highway 280...,Hilton Hampton Inn We’re right off Highway 280...,[ 1.10708795e-01 1.88746721e-01 -1.62566956e-...,"1551 Elkahatchee Road, Alexander City, AL 35010","Alexander City, AL 35010",0.010736
1,1,1,Hilton,Hampton Inn,Hampton Inn Wetumpka,5.0,"We’re on the banks of the Coosa River, a short...",350 South Main Street,Wetumpka,AL,36092,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,Hilton Hampton Inn We’re on the banks of the C...,Hilton Hampton Inn We’re on the banks of the C...,[ 2.21466161e-02 1.88300028e-01 -5.77273928e-...,"350 South Main Street, Wetumpka, AL 36092","Wetumpka, AL 36092",0.005572
2,2,2,Hilton,Hampton Inn,Hampton Inn Auburn,4.0,"We're off I-85, under 10 minutes from Chewacla...",2430 S. College St.,Auburn,AL,36832,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,"Hilton Hampton Inn We're off I-85, under 10 mi...","Hilton Hampton Inn We're off I-85, under 10 mi...",[ 3.20466645e-02 1.81249067e-01 4.03644927e-...,"2430 S. College St., Auburn, AL 36832","Auburn, AL 36832",0.009107
3,3,3,Hilton,Tru,Tru by Hilton Auburn,4.0,"Make this Tru by Hilton your own. We’ll help, ...",2411 W Pace Blvd,Auburn,AL,36830,https://tru3.hilton.com/en/hotels/alabama/tru-...,5,Budget,Hilton Tru Make this Tru by Hilton your own. W...,Hilton Tru Make this Tru by Hilton your own. W...,[ 2.54360028e-02 1.05327815e-01 -9.88091230e-...,"2411 W Pace Blvd, Auburn, AL 36830","Auburn, AL 36830",0.005324
4,4,4,Hilton,Hilton Garden Inn,Hilton Garden Inn Auburn/Opelika,4.0,"Our hotel is just off I-85, minutes away from ...",2555 Hilton Garden Drive,Auburn,AL,36830,https://hiltongardeninn3.hilton.com/en/hotels/...,3,Full-Service Entry,Hilton Hilton Garden Inn Our hotel is just off...,Hilton Hilton Garden Inn Our hotel is just off...,[ 1.27756566e-01 1.37181044e-01 -5.89748239e-...,"2555 Hilton Garden Drive, Auburn, AL 36830","Auburn, AL 36830",0.007964


In [9]:
testdata = get_geocodes(testdata)

0: Latitude = 32.925230083822264, Longitude = -85.96734888230006
1: Latitude = 32.535016163265304, Longitude = -86.20561436734694
2: Latitude = 32.5781087, Longitude = -85.4975499
3: Latitude = 32.5558732, Longitude = -85.50472840014689
4: Latitude = 32.6054196, Longitude = -85.4311496
5: Latitude = 32.6137233, Longitude = -85.4017041
6: Latitude = 32.6137233, Longitude = -85.4017041
7: Latitude = 32.460234, Longitude = -86.390239
8: Latitude = 32.458578, Longitude = -86.4006169
9: Latitude = 34.7809494, Longitude = -86.9370974
10: Latitude = 34.62163121428571, Longitude = -86.99196778571428
11: Latitude = 34.607383, Longitude = -86.979821
12: Latitude = 34.671277236363636, Longitude = -86.76470254545454
13: Latitude = 34.671910999999994, Longitude = -86.7504348231146
14: Latitude = 34.729847, Longitude = -86.5859011
15: Latitude = 34.729847, Longitude = -86.5859011
16: Latitude = 34.594974, Longitude = -87.031232
17: Latitude = 34.729847, Longitude = -86.5859011
18: Latitude = 34.7298

In [10]:
testdata.head()

,Unnamed: 0,Unnamed: 0.1,rewards,brand,name,rating,description,street,city,state,postal,url,class_id,category,Bag_of_words,spacydocs,vectors,address_x,address2,vectormean,address_y,latitude,longitude
0,0,0,Hilton,Hampton Inn,Hampton Inn Alexander City,4.0,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,Hilton Hampton Inn We’re right off Highway 280...,Hilton Hampton Inn We’re right off Highway 280...,[ 1.10708795e-01 1.88746721e-01 -1.62566956e-...,"1551 Elkahatchee Road, Alexander City, AL 35010","Alexander City, AL 35010",0.010736,"1551 Elkahatchee Road, Alexander City, AL 35010",32.925230,-85.967349
1,1,1,Hilton,Hampton Inn,Hampton Inn Wetumpka,5.0,"We’re on the banks of the Coosa River, a short...",350 South Main Street,Wetumpka,AL,36092,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,Hilton Hampton Inn We’re on the banks of the C...,Hilton Hampton Inn We’re on the banks of the C...,[ 2.21466161e-02 1.88300028e-01 -5.77273928e-...,"350 South Main Street, Wetumpka, AL 36092","Wetumpka, AL 36092",0.005572,"350 South Main Street, Wetumpka, AL 36092",32.535016,-86.205614
2,2,2,Hilton,Hampton Inn,Hampton Inn Auburn,4.0,"We're off I-85, under 10 minutes from Chewacla...",2430 S. College St.,Auburn,AL,36832,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Limited-Service Mid-Scale,"Hilton Hampton Inn We're off I-85, under 10 mi...","Hilton Hampton Inn We're off I-85, under 10 mi...",[ 3.20466645e-02 1.81249067e-01 4.03644927e-...,"2430 S. College St., Auburn, AL 36832","Auburn, AL 36832",0.009107,"2430 S. College St., Auburn, AL 36832",32.578109,-85.497550
3,3,3,Hilton,Tru,Tru by Hilton Auburn,4.0,"Make this Tru by Hilton your own. We’ll help, ...",2411 W Pace Blvd,Auburn,AL,36830,https://tru3.hilton.com/en/hotels/alabama/tru-...,5,Budget,Hilton Tru Make this Tru by Hilton your own. W...,Hilton Tru Make this Tru by Hilton your own. W...,[ 2.54360028e-02 1.05327815e-01 -9.88091230e-...,"2411 W Pace Blvd, Auburn, AL 36830","Auburn, AL 36830",0.005324,"2411 W Pace Blvd, Auburn, AL 36830",32.555873,-85.504728
4,4,4,Hilton,Hilton Garden Inn,Hilton Garden Inn Auburn/Opelika,4.0,"Our hotel is just off I-85, minutes away from ...",2555 Hilton Garden Drive,Auburn,AL,36830,https://hiltongardeninn3.hilton.com/en/hotels/...,3,Full-Service Entry,Hilton Hilton Garden Inn Our hotel is just off...,Hilton Hilton Garden Inn Our hotel is just off...,[ 1.27756566e-01 1.37181044e-01 -5.89748239e-...,"2555 Hilton Garden Drive, Auburn, AL 36830","Auburn, AL 36830",0.007964,"2555 Hilton Garden Drive, Auburn, AL 36830",32.605420,-85.431150
